<a href="https://colab.research.google.com/github/HassanDataSci/caricon-personality-career-matching/blob/data-standardization/CariCon_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

# Load each file into its own DataFrame
df_mbti   = pd.read_excel("data/myer-briggs-data.xlsx")
df_onet  = pd.read_excel("data/complete_onet_data_with_human_characteristics.xlsx")

print("MBTI shape:", df_mbti.shape)
print("Human characteristics shape:", df_onet.shape)

MBTI shape: (8675, 2)
Human characteristics shape: (1016, 30)


In [18]:
print("=== MBTI Data ===")
print(df_mbti.dtypes)
print()

print("=== O*NET Human Characteristics ===")
print(df_onet.dtypes)
print()

=== MBTI Data ===
type     object
posts    object
dtype: object

=== O*NET Human Characteristics ===
Occupation Code             object
Title                       object
Human Characteristics       object
Description                 object
Tasks                       object
Knowledge                   object
Skills                      object
Abilities                   object
Work Activities             object
Detailed Work Activities    object
Technology Used             object
Job Zone                    object
Education Level             object
Experience Required         object
Job Training                object
SVP Range                   object
Job Zone Examples           object
Interests                   object
Work Styles                 object
Work Values                 object
Work Context                object
Related Occupations         object
dtype: object



# Handling Missing Data

In [4]:
nan_count = np.sum(df_onet.isnull())
print(nan_count)

Occupation Code               0
Title                         0
Human Characteristics         0
Description                   0
Sample Job Titles           115
Tasks                        93
Knowledge                   143
Skills                      147
Abilities                   138
Work Activities             137
Detailed Work Activities     93
Tools Used                  119
Technology Used              93
Job Zone                     93
Education Level              93
Experience Required          93
Job Training                 93
SVP Range                    93
Job Zone Examples            93
Interests                    93
Work Styles                 137
Work Values                 142
Work Context                137
Additional Sources           67
Related Occupations          93
Annual 10th Percentile       26
Annual 25th Percentile       26
Annual Median Wage           26
Annual 75th Percentile       26
Annual 90th Percentile       26
dtype: int64


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [5]:
# many of the missing values come from rows with "All Other" in the title
df_onet['Title'].str.contains("All Other", case=False).value_counts()

,count
Title,
False,939
True,77


In [6]:
# So we can drop rows with all other
df_onet = df_onet[~df_onet['Title'].str.contains("All Other", case=False)]
df_onet.shape

(939, 30)

In [7]:
nan_count = np.sum(df_onet.isnull())
print(nan_count)

Occupation Code              0
Title                        0
Human Characteristics        0
Description                  0
Sample Job Titles           38
Tasks                       16
Knowledge                   66
Skills                      70
Abilities                   61
Work Activities             60
Detailed Work Activities    16
Tools Used                  42
Technology Used             16
Job Zone                    16
Education Level             16
Experience Required         16
Job Training                16
SVP Range                   16
Job Zone Examples           16
Interests                   16
Work Styles                 60
Work Values                 65
Work Context                60
Additional Sources          17
Related Occupations         16
Annual 10th Percentile      22
Annual 25th Percentile      22
Annual Median Wage          22
Annual 75th Percentile      22
Annual 90th Percentile      22
dtype: int64


In [8]:
# 16 of the same rows all have missing values across mutiple columns, these are primarily military roles
df_onet[df_onet['Technology Used'].isnull()]['Title']


,Title
22,Air Crew Members
23,Air Crew Officers
26,Aircraft Launch and Recovery Officers
27,Aircraft Launch and Recovery Specialists
54,Armored Assault Vehicle Crew Members
55,Armored Assault Vehicle Officers
59,Artillery and Missile Crew Members
60,Artillery and Missile Officers
167,Command and Control Center Officers
168,Command and Control Center Specialists


In [9]:
# so we can drop those roles
df_onet = df_onet[~df_onet['Technology Used'].isnull()]

nan_count = np.sum(df_onet.isnull())
print(nan_count)

Occupation Code              0
Title                        0
Human Characteristics        0
Description                  0
Sample Job Titles           22
Tasks                        0
Knowledge                   50
Skills                      54
Abilities                   45
Work Activities             44
Detailed Work Activities     0
Tools Used                  26
Technology Used              0
Job Zone                     0
Education Level              0
Experience Required          0
Job Training                 0
SVP Range                    0
Job Zone Examples            0
Interests                    0
Work Styles                 44
Work Values                 49
Work Context                44
Additional Sources           1
Related Occupations          0
Annual 10th Percentile       6
Annual 25th Percentile       6
Annual Median Wage           6
Annual 75th Percentile       6
Annual 90th Percentile       6
dtype: int64


In [10]:
# Drop some irrelevant columns that we won't use as predictors
df_onet.drop(columns=['Tools Used','Sample Job Titles','Additional Sources'], inplace=True)
df_onet.drop(df_onet.columns[-5:], axis=1, inplace=True)

In [11]:
nan_count = np.sum(df_onet.isnull())
print(nan_count)

Occupation Code              0
Title                        0
Human Characteristics        0
Description                  0
Tasks                        0
Knowledge                   50
Skills                      54
Abilities                   45
Work Activities             44
Detailed Work Activities     0
Technology Used              0
Job Zone                     0
Education Level              0
Experience Required          0
Job Training                 0
SVP Range                    0
Job Zone Examples            0
Interests                    0
Work Styles                 44
Work Values                 49
Work Context                44
Related Occupations          0
dtype: int64


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


These are the roles that still have missing data in columns that we care about, and it's a total of 65 roles. To fill in these missing values, we would have to go online and manually search for skills, abilities, work values, etc.

It could be worth it to spend time doing so in the future. But for now, I think it's more efficient to just drop these roles.

In [12]:
roles_missing_vals = df_onet[df_onet.isnull().any(axis=1)]['Title'].to_list()
roles_missing_vals

['Aircraft Service Attendants',
 'Appraisers of Personal and Business Property',
 'Blockchain Engineers',
 'Bus Drivers, School',
 'Calibration Technologists and Technicians',
 'Cardiologists',
 'Clinical Neuropsychologists',
 'Crematory Operators',
 'Cutters and Trimmers, Hand',
 'Data Scientists',
 'Digital Forensics Analysts',
 'Disc Jockeys, Except Radio',
 'Dishwashers',
 'Electrical and Electronic Equipment Assemblers',
 'Emergency Medical Technicians',
 'Emergency Medicine Physicians',
 'Entertainment and Recreation Managers, Except Gambling',
 'Facilities Managers',
 'Fallers',
 'Farmworkers and Laborers, Crop, Nursery, and Greenhouse',
 'Financial and Investment Analysts',
 'Financial Risk Specialists',
 'First-Line Supervisors of Entertainment and Recreation Workers, Except Gambling Services',
 'First-Line Supervisors of Passenger Attendants',
 'First-Line Supervisors of Security Workers',
 'Fundraising Managers',
 'Graders and Sorters, Agricultural Products',
 'Health Inform

In [13]:
df_onet = df_onet[~df_onet['Title'].isin(roles_missing_vals)]
np.sum(df_onet.isnull()).sum()

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


np.int64(0)

# Data Standardize

In [14]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

In [19]:
# Select categorical and numerical columns
categorical_cols = df_onet.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df_onet.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Categorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)

Categorical columns: ['Occupation Code', 'Title', 'Human Characteristics', 'Description', 'Tasks', 'Knowledge', 'Skills', 'Abilities', 'Work Activities', 'Detailed Work Activities', 'Technology Used', 'Job Zone', 'Education Level', 'Experience Required', 'Job Training', 'SVP Range', 'Job Zone Examples', 'Interests', 'Work Styles', 'Work Values', 'Work Context', 'Related Occupations']
Numerical columns: []


In [20]:
'''#Convert columns to numerical due to NaN
num_cols_to_convert = [
    'Annual 10th Percentile',
    'Annual 25th Percentile',
    'Annual Median Wage',
    'Annual 75th Percentile',
    'Annual 90th Percentile'
]

for col in num_cols_to_convert:
    df_onet[col] = pd.to_numeric(df_onet[col], errors='coerce')'''

"#Convert columns to numerical due to NaN\nnum_cols_to_convert = [\n    'Annual 10th Percentile',\n    'Annual 25th Percentile',\n    'Annual Median Wage',\n    'Annual 75th Percentile',\n    'Annual 90th Percentile'\n]\n\nfor col in num_cols_to_convert:\n    df_onet[col] = pd.to_numeric(df_onet[col], errors='coerce')"

In [22]:
#one-hot encoding for categorical columns
df_onet_encoded = pd.get_dummies(df_onet, columns=categorical_cols, drop_first=True)
print("Shape after encoding:", df_onet_encoded.shape)

Shape after encoding: (858, 10949)


In [24]:
'''#Min-Max scaling for numerical columns
scaler = MinMaxScaler()
df_onet_encoded[numerical_cols] = scaler.fit_transform(df_onet_encoded[numerical_cols])'''

'#Min-Max scaling for numerical columns\nscaler = MinMaxScaler()\ndf_onet_encoded[numerical_cols] = scaler.fit_transform(df_onet_encoded[numerical_cols])'

In [28]:
bool_cols = df_onet_encoded.select_dtypes(include='bool').columns
df_onet_encoded[bool_cols] = df_onet_encoded[bool_cols].astype(int)

In [29]:
df_onet_encoded.iloc[:10, :20]

,Occupation Code_11-1011.03,Occupation Code_11-1021.00,Occupation Code_11-2011.00,Occupation Code_11-2021.00,Occupation Code_11-2022.00,Occupation Code_11-3012.00,Occupation Code_11-3021.00,Occupation Code_11-3031.00,Occupation Code_11-3031.01,Occupation Code_11-3031.03,Occupation Code_11-3051.00,Occupation Code_11-3051.01,Occupation Code_11-3051.02,Occupation Code_11-3051.03,Occupation Code_11-3051.04,Occupation Code_11-3051.06,Occupation Code_11-3061.00,Occupation Code_11-3071.00,Occupation Code_11-3071.04,Occupation Code_11-3111.00
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
